# BDSN ASSIGNMENT
# ISHAAN NIRMAL - A21015

 **About the code**
 
 Basically what we do here is examine a dataset with Ecommerce Customer Data for a company's website and mobile app,then we build a linear regression model that will predict the customer's yearly spend on the company's product.

**Installing spark**

In [77]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://apache.osuosl.org/spark/spark-2.2.2/spark-2.2.2-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
#wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
#!wget -q http://apache.osuosl.org/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#
#!tar xf spark-2.4.0-bin-hadoop2.7.tgz
#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
#!tar xf spark-2.4.6-bin-hadoop2.7.tgz
#!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark
!pip install -q pyspark

In [78]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [79]:
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [80]:
import warnings
warnings.filterwarnings("ignore")

# Loading the data 

In [81]:
data = spark.read.csv("/content/Ecommerce_Customers.csv",header = True , inferSchema = True)

In [82]:
data

DataFrame[Email: string, Address: string, Avatar: string, Avg Session Length: double, Time on App: double, Time on Website: double, Length of Membership: double, Yearly Amount Spent: double]

In [83]:
data.printSchema()
## This shows the schema of the dataset

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [84]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [104]:
data.describe().show()

+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|summary|            Email|             Address|     Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----------------+--------------------+-----------+------------------+------------------+------------------+--------------------+-------------------+
|  count|              500|                 500|        500|               500|               500|               500|                 500|                500|
|   mean|             null|                null|       null| 33.05319351819619|12.052487937166134| 37.06044542094859|   3.533461555915055|  499.3140382585909|
| stddev|             null|                null|       null|0.9925631110845354|0.9942156084725424|1.0104889067564033|  0.9992775024112585|   79.3147815497068|
|    min|aaron04@yahoo.com|0001 Mack MillNor..

**Modelling Part**

In [86]:
## Now we will setup the dataframe for spark
## Spark accepts the data frame in the form of ('labels','features')
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [87]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [105]:
assembler = VectorAssembler(inputCols=["Avg Session Length", "Time on App", "Time on Website",'Length of Membership'],outputCol="features")
## Splitting the data into labels & features

In [106]:
output = assembler.transform(data)

In [107]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [108]:
## Displaying the output with the feature column now 
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [109]:
new_data = output.select("features","Yearly Amount Spent")

In [110]:
## Splitting the data into train & test 
X_train , Y_test = new_data.randomSplit([0.7,0.3])

In [111]:
X_train.cache()
Y_test.cache()

DataFrame[features: vector, Yearly Amount Spent: double]

In [112]:
X_train.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                358|
|   mean| 502.01951451152667|
| stddev|  82.17590366737566|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [113]:
Y_test.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                142|
|   mean|  492.4931896772456|
| stddev|  71.41536554117073|
|    min|  319.9288698031936|
|    max|  744.2218671047146|
+-------+-------------------+



In [114]:
from pyspark.ml.regression import LinearRegression
## Creating the object for the model 
lr = LinearRegression(labelCol="Yearly Amount Spent")

In [115]:
## Now we are fitting the model to the data 
lrmodel = lr.fit(X_train,)

In [120]:
test_results = lrmodel.evaluate(Y_test)
test_results.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
|-11.749119631811368|
| 10.082969169371268|
| -5.055061001823049|
| -4.426315409544657|
|  6.419910712063029|
+-------------------+
only showing top 5 rows



In [121]:
unlabel_data = Y_test.select('features')

In [122]:
pred = lrmodel.transform(unlabel_data)
pred.show()
## Displaying the predictions done 
## These predictions are the people's yearly amount spent on the company's products 

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|  331.677989435005|
|[30.7377203726281...| 451.6977730268586|
|[30.8364326747734...|472.55696142881266|
|[30.8794843441274...|494.63291539439933|
|[30.9716756438877...| 488.2186990448297|
|[31.0472221394875...| 387.2553765225696|
|[31.1280900496166...| 565.2321589983799|
|[31.3584771924370...| 491.8016397350764|
|[31.3662121671876...|426.63603770919144|
|[31.5261978982398...| 417.7954205931435|
|[31.6098395733896...| 428.0958187366334|
|[31.7207699002873...| 545.5527130800422|
|[31.7216523605090...|349.59568889321895|
|[31.7242025238451...| 510.4423322498967|
|[31.7366356860502...|494.74412382541254|
|[31.8124825597242...| 396.7653634151186|
|[31.8186165667690...| 448.9703544127092|
|[31.8648325480987...| 450.9753141611334|
|[31.8745516945853...| 397.6600699859639|
|[31.9262720263601...| 380.0639871438666|
+--------------------+------------

In [123]:
## co-efficients & intercepts 
print("Coefficients: {} Intercept: {}".format(lrmodel.coefficients,lrmodel.intercept))

Coefficients: [25.828386753145224,39.08699039477006,0.12922700976810583,61.65497595629178] Intercept: -1047.8426186900226


In [124]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.19985715182144
MSE: 84.6373716139201
